## Reward function

In [ ]:
import puzzle

def reward(self):
    increment = self.game.increment
    totalScore = self.game.totalScore
    # loose does not work yet
    #loose = self.game.loose
    return increment

## Neural Network Model

In [ ]:
import random
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=100000)
        self.gamma = 0.9    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.e_decay = .99
        self.e_min = 0.05
        self.learning_rate = 0.01
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(20, input_dim=self.state_size, activation='tanh')) # Adds the first layer with 16 inputs
        model.add(Dense(20, activation='tanh', init='uniform')) # Adds Hidden layer with 20 nodes
        model.add(Dense(self.action_size, activation='linear')) # Adds output layer with 2 nodes
        model.compile(loss='mse',
                      optimizer=RMSprop(lr=self.learning_rate)) # Creates the model from all of the above
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        batch_size = min(batch_size, len(self.memory))
        minibatch = random.sample(self.memory, batch_size)
        X = np.zeros((batch_size, self.state_size))
        Y = np.zeros((batch_size, self.action_size))
        for i in range(batch_size):
            state, action, reward, next_state, done = minibatch[i]
            target = self.model.predict(state)[0]
            if done:
                target[action] = reward
            else:
                target[action] = reward + self.gamma * \
                            np.amax(self.model.predict(next_state)[0])
            X[i], Y[i] = state, target
        self.model.fit(X, Y, batch_size=batch_size, nb_epoch=1, verbose=0)
        if self.epsilon > self.e_min:
            self.epsilon *= self.e_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

## Main Game execution

In [ ]:
import time
import puzzle
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop
import numpy as np

EPISODES = 10

#ACCESS SCORE AS self.game.score
#ACCESS MATRIX AS self.game.matrix
#DECIDE ACTION TO TAKE IN act()
#POSSIBLE ACTIONS:
#	go up:		"'w'"
#	go left:	"'a'"
#	go right:	"'s'"
#	go down:	"'d'"

class Machine:
    game=puzzle.GameGrid() # Game object
    model = Sequential()
    inputVector=np.zeros((1, 16))
    Qvalues0=np.zeros((1,4))
    Qvalues1=np.zeros((1,4))
    acts = ["'w'","'s'","'d'","'a'"]
    gamma = 0.9    # Discount rate
    def __init__(self):
        # Create model
        self.model.add(Dense(20, input_dim=16, activation='tanh')) # Adds the first layer with 16 inputs
        self.model.add(Dense(20, activation='tanh'))              # Adds Hidden layer with 20 nodes (Removed 'uniform')
        self.model.add(Dense(4, activation='linear'))             # Adds output layer with 20 nodes
        self.model.compile(loss='mse',optimizer=RMSprop(lr=0.01)) # Creates the model from all of the above
    def run(self):
        # Transform game state to 1D array
        for i in range(4):
            self.inputVector[0][0+4*i:4+4*i]=self.game.matrix[i]
        self.game.increment=self.reward() # Update reward if game has been lost
        self.game.key_down(self.act()) # Select action and update weights
        # Game loop
        self.game.update_idletasks
        self.game.update()
        # pool emaG
    def act(self):
        # Predict Q values of current state
        self.Qvalues1[0]=self.gamma*self.model.predict(self.inputVector)+self.game.increment
        # Update weights with respect to last step's prediction of this step's Q values
        self.model.train_on_batch(self.inputVector, self.Qvalues0)
        # Make this step's Q values next step's past Q values
        self.Qvalues0=self.Qvalues1
        # Select action with highest Q value
        return self.acts[self.Qvalues1.argmax()]
        
    def reward(self):
        l=0
        if self.game.result: 
            l = -2*self.game.increment-self.game.score/4
            print(self.game.score)
            self.game.reset()
        return (self.game.increment + l)
    
    #######################################
    # I made this function to calculate the best next step. This will be the function
    # Determening which state to be given to the neural network.
    # This is a bad implementation since it relies on copying the whole game for every
    # calculation, but it will work to train in the beginning. 
    def calculateNextStep(self, game):
        best_qval = 0
        best_move = 'up' # Default move
        acts = ['up','down','right','left']
        for act in acts: 
            game_copy = game
            game_copy.key_down(self.act(act))
            self.game.update_idletasks
            self.game.update()
            qval = game_copy.reward()
            if qval > best_qval:
                best_qval = qval
                best_move = act
        return best_move
    #######################################
        
our_machine = Machine()
while True:
    our_machine.run()

Using TensorFlow backend.


468
1264
952
508
312
736
896
972
340
460
388
500
732
284
1232
596
384
676
680
420
376
228
400
648
796
548
1236
876
808
976
456
328
884
476
824
1388
988
428
392
1088
572
536
428
692
560
1524
516
532
524
356
688
384
600
484
576
620
340
864
1244
764
280
656
564
252
960
596
1496
688
832
380
500
976
660
968
1408
548
892
504
520
784
512
480
724
800
448
512
564
468
816
1068
516
720
1384
628
996
700
320
860
560
924
876
780
1188
92
324
940
1104
720
600
1448
640
684
600
572
1024
396
752
604
656
396
264
788
1192
636
364
1080
572
588
1452
416
508
484
1352
464
508
440
400
664
708
448
516
656
324
1044
452
1108
1728
596
516
1104
468
504
536
988
792
1136
520
612
1352
368
408
716
1020
580
548
1256
480
924
504
540
500
644
792
332
436
928
420
928
560
652
380
768
556
924
532
392
756
764
292
692
984
640
816
368
544
940
1008
432
776
536
1248
380
780
372
844
532
492
380
276
552
540
960
976
504
808
616
692
696
732
916
964
744
316
432
336
392
948
344
220
388
748
452
296
696
580
812
1184
228
792


In [ ]:
# Easier implemtation to start with. I have used this guide: https://keon.io/deep-q-learning/
# The following code is to see that the neural network and deep q network is setup correctly

# 1
############## Simple Neural Net #################
# Here are some more concrete numbers to put into the neural network than in the DQNAgent
model = Sequential()
model.add(Dense(20, input_dim=4, activation='tanh')) # Adds the first layer with 16 inputs
model.add(Dense(20, activation='tanh'))              # Adds Hidden layer with 20 nodes (Removed 'uniform')
model.add(Dense(4, activation='linear'))             # Adds output layer with 2 nodes
model.compile(loss='mse',optimizer=RMSprop(lr=0.01)) # Creates the model from all of the above

#     This training process makes the neural net predict 
#     the reward value (target_f) from a certain state.

state = our_machine.game.matrix  # Game state matrix ( Converting to a 1D array to please model.fit())                      
print(state)


# After some debugging the target has to be a table just like the state
# Therefore I am wondering if we should have the target as the complete theoretical board?
# Or as the best possible next state? 

# Theoretical limit target_f = [[131072, 65536, 32768, 16384], [1024, 2048, 4096, 8192], [512, 256, 128, 64], [4, 8, 16, 32]]
# I we want to use the best possible next state we first have to: 
    # 1: Calculate the next four possibilities
    # 2: Pick the one with highest q-value
    # 3: Use that next state as target_f
    
# For now I will use this random state
target_f = [[0, 0, 2, 0], [0, 0, 0, 0], [0, 0, 2, 0], [0, 0, 0, 0]]

model.fit(state, target_f)        # Fitting the data (Feeding the network)
prediction = model.predict(state) # Predicting the reward from the current state



# 2
############## Deep Q Network (DQN) #################

# So that the neural network don't "forget" the old state with new experiences
# we make a function to remember. (Already implemented perfectly in DQNAgent)
memory = ((state, action, reward, next_state, done))

def remember(self, state, action, reward, next_state, done):
    self.memory.append((state, action, reward, next_state, done))

# Replay trains the neural network by replaying the previous experiences in its memory
# These memories are experienced in batches

batch_size = 5 # Just a random number 
batches = min(batch_size, len(self.memory))
batches = np.random.choice(len(self.memory), batches) # Picks random experiences to make batches of


# Training
    
        

In [ ]:
import numpy as np

a=np.array([[1,2,7],[3,4,5]])
b=np.array([0,0,0,0,0,0])
for i in range(2):
    b[0+3*i:3+3*i]=a[i]
print(b[b.argmax()])
print(b.shape)
x_test = np.zeros((1, 16))
print(x_test)